In [180]:
import pandas as pd
import numpy as np
import json
import ast


df = pd.read_csv('Kickstarter001.csv')
df.head()
features = df.columns[18:]
df1 = df[features]

photo_link = []
for i in range(len(df1)):
    photo_dict =df1.iloc[i].photo
    photo_link.append(ast.literal_eval(p)['full'])
    
country = []
state = []
town = []
loc_type = []
for i in range(len(df1)):
    try:
        loc_dict = json.loads(df1.iloc[i].location)
        country.append(str(loc_dict['country']))
    except Exception as e:
        country.append("None")
for i in range(len(df1)):
    try:
        loc_dict = json.loads(df1.iloc[i].location)
        state.append(str(loc_dict['state']))
    except Exception as e:
        state.append("None")
for i in range(len(df1)):
    try:
        loc_dict = json.loads(df1.iloc[i].location)
        town.append(str(loc_dict['name']))
    except Exception as e:
        town.append("None")
for i in range(len(df1)):
    try:
        loc_dict = json.loads(df1.iloc[i].location)
        loc_type.append(str(loc_dict['type']))
    except Exception as e:
        loc_type.append("None")

df1['country'] = np.array(country)
df1['state'] = np.array(state)
df1['town'] = np.array(town)
df1['location_type'] = np.array(loc_type)

project_url = []
reward_url =[]
for i in range(len(df1)):
    try:
        project_url.append(str(json.loads(df1.iloc[0]['urls'])['web']['project']))
        reward_url.append(str(json.loads(df1.iloc[0]['urls'])['web']['rewards']))
    except Exception as e:
        country.append("None")
        
df1['project_url'] = project_url
df1['reward_url'] = reward_url

df1.drop(['urls','photo','location','profile'],axis=1,inplace=True)
df1.head()

/Users/saurabh/venv/lib/python2.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/saurabh/venv/lib/python2.7/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/saurabh/venv/lib/python2.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/index

,name,pledged,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,usd_pledged,usd_type,friends,is_backing,is_starred,permissions,country,town,location_type,project_url,reward_url
0,The Curious Concepts Project,634.0,the-curious-concepts-project,https://www.kickstarter.com/discover/categorie...,True,False,NY,1367651037,1.000000,634.000000,domestic,NaN,NaN,NaN,NaN,US,Lockport,Town,https://www.kickstarter.com/projects/690850738...,https://www.kickstarter.com/projects/690850738...
1,"E.T. The Extra-Terrestrial Atari ""30th Anniver...",1016.0,et-the-extra-terrestrial-30th-anniversary-poster,https://www.kickstarter.com/discover/categorie...,True,False,TN,1367283633,1.000000,1016.000000,domestic,NaN,NaN,NaN,NaN,US,Nashville,Town,https://www.kickstarter.com/projects/690850738...,https://www.kickstarter.com/projects/690850738...
2,Iconic New Orleans Fine Art Prints,945.0,iconic-new-orleans-fine-art-prints,https://www.kickstarter.com/discover/categorie...,True,False,LA,1368040821,1.000000,945.000000,domestic,NaN,NaN,NaN,NaN,US,New Orleans,Town,https://www.kickstarter.com/projects/690850738...,https://www.kickstarter.com/projects/690850738...
3,Face of a DREAM...,5949.0,face-of-a-dream-0,https://www.kickstarter.com/discover/categorie...,True,False,MO,1368881143,1.000000,5949.000000,domestic,NaN,NaN,NaN,NaN,US,Kansas City,Town,https://www.kickstarter.com/projects/690850738...,https://www.kickstarter.com/projects/690850738...
4,GO ZOMBIE! Zombie Portrait Print Project.,3662.0,go-zombie-zombie-portrait-print-project,https://www.kickstarter.com/discover/categorie...,True,False,England,1369699038,1.546855,5664.583047,domestic,NaN,NaN,NaN,NaN,GB,North Yorkshire,County,https://www.kickstarter.com/projects/690850738...,https://www.kickstarter.com/projects/690850738...


In [181]:
df1.to_csv('c')

<bound method DataFrame.to_csv of                                                    name   pledged  \
0                          The Curious Concepts Project    634.00   
1     E.T. The Extra-Terrestrial Atari "30th Anniver...   1016.00   
2                    Iconic New Orleans Fine Art Prints    945.00   
3                                    Face of a DREAM...   5949.00   
4             GO ZOMBIE! Zombie Portrait Print Project.   3662.00   
5             Denver ComicCon Print by Sobol and Kordos    561.00   
6     Pet Portraits! Immortalize your Best Friend in...   3526.00   
7                    Lucid dream: Fantasy portrait art.    370.00   
8                EclecDeck Transformation Playing Cards   5205.00   
9     The Sportsmen of Oz Limited Edition Trading Ca...   1396.00   
10    "A Fiend In Need" Art Print - 80s Metal x Dogs...   2179.00   
11                                        Pet Portraits  13817.00   
12         365 2 40 Illustration Project & Gallery Show   4025.00   
